# Audio Steganography

In [8]:
import wave

In [9]:
def encode(input_audio, output_audio, message):
  audio_file = wave.open(input_audio, mode ='rb')

  # read frames of the audio and convert it to byte array
  frames = audio_file.readframes(-1)
  byte_array = bytearray(frames)

  # convert the message to binary
  message = [int(i) for i in ''.join([bin(ord(c)).lstrip('0b').rjust(8,'0') for c in message])]

  # convert message length to binary and make it of 32 bits
  message_len = len(message)
  message_len_bin = bin(message_len)[2:].zfill(32)

  # put message length information in first 32 bits of audio
  message_len_bin = list(message_len_bin)
  for i in range(0,32):
    byte_array[i] = ((byte_array[i] & 254) | int(message_len_bin[i]))

  # put secret message in LSB of audio bytes
  for i, bit in enumerate(message):
    byte_array[i+32] = (byte_array[i+32] & 254) | bit

  # create audio file with modified frames
  frame_modified = bytes(byte_array)
  with wave.open(output_audio, 'wb') as fd:
    fd.setparams(audio_file.getparams())
    fd.writeframes(frame_modified)

  audio_file.close()

In [10]:
def decode(encoded_audio):
  audio_file = wave.open(encoded_audio, mode ='rb')

  # read frames of the audio and convert it to byte array
  frames = audio_file.readframes(-1)
  byte_array = bytearray(frames)

  # find message length information from first 32 bits of audio
  message_len_bin = ""
  for i in range(0,32):
      bit = bin(byte_array[i])[2:].zfill(8)
      message_len_bin += str(bit[-1])

  # Extract message from the LSB of audio byte array
  message = []
  message_len = int(message_len_bin,2)
  for i in range(message_len):
      message.append(byte_array[i+32] & 1)

  # Convert the decoded message in binary to string
  decoded_msg = ''.join([chr(int(i, 2)) for i in [''.join(map(str,message[i:i+8])) for i in range(0, len(message), 8)]])

  audio_file.close()

  return decoded_msg

## Encode here

In [11]:
input_path = input("Enter input audio file path: ")
output_path = input("Enter the output audio file path: ")
message = input("Enter the secret text to hide in audio file: ")
encode(input_path, output_path, message)
print("Message encoded")

Enter input audio file path: /content/cover_audio.wav
Enter the output audio file path: /content/cover_audio_output.wav
Enter the secret text to hide in audio file: Himanshu
Message encoded


## Decode here

In [12]:
output_path = input("Enter the audio file path to decode message: ")
decoded_msg = decode(output_path)
print("Message decoded: ", decoded_msg)

Enter the audio file path to decode message: /content/cover_audio_output.wav
Message decoded:  Himanshu
